In [1]:
## Make gCode colorful

import pandas as pd
import numpy as np
import copy
from gcodeparser import GcodeParser, GcodeLine
from datetime import datetime

In [ ]:
class ColorChange:

    def __init__(self, index, gcode_line_from_df, order=0):
        self.index = int(index)
        self.order = int(order)
        self.gcode_line_from_df = gcode_line_from_df

class ColorChangeHandler:
    
    def __init__(self, gcode_container, color_change_order, reduce_pressure_filament_distance, negative_pressure_reduction_factor=1, rectraction_length=None, retraction_speed=None, refill_retraction_length=None):

        self.gcode_container = gcode_container
        self.gcode_df = gcode_container.gcode_df
        self.gcode_df_color = None

        self.color_change_order = color_change_order

        self.color_changes = []
        
        # retraction
        self.rectraction_length = rectraction_length
        if refill_retraction_length is None:
            self.refill_retraction_length = rectraction_length
        else:
            self.refill_retraction_length = refill_retraction_length
        self.retraction_speed = retraction_speed

        # pressure control color change
        self.reduce_pressure_filament_distance = reduce_pressure_filament_distance
        self.negative_pressure_reduction_factor = negative_pressure_reduction_factor

        self.gcode_result = None
        self.type_of_change_label_list = []


    @staticmethod
    def get_color_change_commands(color_param):
    
        color_change_commands = [
                                    GcodeLine(("M", 163), {"S": 0, "P": color_param[0]}, comment=""),
                                    GcodeLine(("M", 163), {"S": 1, "P": color_param[1]}, comment=""),
                                    GcodeLine(("M", 163), {"S": 2, "P": color_param[2]}, comment=""),
                                    GcodeLine(("M", 164), {"S": 0}, comment="")]               

        return color_change_commands

    def adjust_pressure_around_color_change(self, color_change, gcode_raw):
       
        # scan for gcode lines for pressure reduction
        # 1. first reverse
        reduction_distance = 0
        for i in range(200):
            gcode_index = color_change.index - i - 1
            line = gcode_raw[gcode_index]
            if "E" not in line.params:
                print("NO E", line.gcode_str)
                e_len = 0
            else:
                e_len = line.params["E"]

            if line.command != ("G", 1):
                print("Warning adjust pressure no G1: {}".format(line.gcode_str))
            else:

                start_e = reduction_distance
                center_e = start_e + 0.5 * e_len

                center_percent = center_e / self.reduce_pressure_filament_distance

                e_len_new = e_len * center_percent 

                #print("- Dist: {} | Line: {} elen: {} -> {} | percent: {}".format(reduction_distance, gcode_index, e_len, e_len_new, center_percent))

                # set new e_len
                gcode_raw[gcode_index].params["E"] = e_len_new
                gcode_raw[gcode_index].comment = "PressureControlFactor: {}".format(center_percent)

                reduction_distance += e_len

            if reduction_distance >= self.reduce_pressure_filament_distance:
                break
        # 2. forward
        accel_distance = 0
        for i in range(200):
            gcode_index = color_change.index + i 
            
            if gcode_index >= len(gcode_raw):
                break

            line = gcode_raw[gcode_index]
            
            if "E" not in line.params:
                print("NO E", line.gcode_str)
                e_len = 0
            else:
                e_len = line.params["E"]

            if line.command != ("G", 1):
                print("Warning adjust pressure no G1: {}".format(line.gcode_str))
            else:

                start_e = accel_distance
                center_e = start_e + 0.5 * e_len

                center_percent = 2 - (center_e / self.reduce_pressure_filament_distance)

                e_len_new = e_len * center_percent

                #print("+ Dist: {} | Line: {} elen: {} -> {} | percent: {}".format(accel_distance, gcode_index, e_len, e_len_new, center_percent))

                # set new e_len
                gcode_raw[gcode_index].params["E"] = e_len_new
                gcode_raw[gcode_index].comment = "PressureControlFactor: {}".format(center_percent)
                accel_distance += e_len

            if accel_distance >= self.reduce_pressure_filament_distance:
                break
            
        return gcode_raw


    def add_retraction_commands(self, color_change_commands, layer_nr):

        # rectraction_length
        if isinstance(self.rectraction_length, list):
            number_of_sections = len(self.rectraction_length)
            layers_per_section = int(self.gcode_container.model_layer_count / number_of_sections)
            section_index = layer_nr // layers_per_section
            #print("line -> section", layer_nr, section_index)
            rectraction_length_loc = self.rectraction_length[section_index]
            
        else:
            rectraction_length_loc = self.rectraction_length

        # rectraction_length
        if isinstance(self.refill_retraction_length, list):
            number_of_sections = len(self.refill_retraction_length)
            layers_per_section = int(self.gcode_container.model_layer_count / number_of_sections)
            section_index = layer_nr // layers_per_section
            refill_retraction_length_loc = self.refill_retraction_length[section_index]
        else:
            refill_retraction_length_loc = self.refill_retraction_length

        # rectraction_length
        if isinstance(self.retraction_speed, list):
            number_of_sections = len(self.retraction_speed)
            layers_per_section = int(self.gcode_container.model_layer_count / number_of_sections)
            section_index = layer_nr // layers_per_section
            retraction_speed_loc = self.retraction_speed[section_index]
        else:
            retraction_speed_loc = self.retraction_speed

        if rectraction_length_loc != 0:
            color_change_commands = [GcodeLine(("G", 1), {"E": -rectraction_length_loc, "F": retraction_speed_loc}, comment="")] + color_change_commands

            color_change_commands += [GcodeLine(("G", 1), {"E": refill_retraction_length_loc, "F": retraction_speed_loc}, comment="")]

        return color_change_commands
 
    def calc_color_changes(self, gcode_df_color, color_val_label):
        gcode_df_color["color_change_index"] = gcode_df_color["delta_val"].diff().abs()

        self.gcode_df_color = gcode_df_color

        # find colorChangeIndex
        color_change_lines = gcode_df_color[gcode_df_color["color_change_index"] == 1]
        color_changes = [ColorChange(index=line.line_nr, gcode_line_from_df=line) for _, line in color_change_lines.iterrows()]

        self.color_changes += color_changes

        print(color_val_label)
        #display(gcode_df_color[[color_val_label, "delta_val", "delta", "color_change_index"]])
    
    def calc_color_change_index_by_layer_count(self, delta_layer_count):
        color_val_label = "layer"
        self.type_of_change_label_list.append("layer")
        gcode_df_color = self.gcode_df.copy()
        color_change_val = gcode_df_color["layer"]
        gcode_df_color["delta"] = color_change_val % delta_layer_count
        gcode_df_color["delta_val"] = color_change_val // delta_layer_count

        self.calc_color_changes(gcode_df_color, color_val_label)

    def calc_color_change_index_by_distance_xy(self, delta_distance_mm):
        color_val_label = "distance_xy_cumsum"
        self.type_of_change_label_list.append("distance")
        gcode_df_color = self.gcode_df.copy()
        color_change_val = gcode_df_color["distance_xy_cumsum"]
        gcode_df_color["delta"] = color_change_val % delta_distance_mm
        gcode_df_color["delta_val"] = color_change_val // delta_distance_mm

        self.calc_color_changes(gcode_df_color, color_val_label)

    def calc_color_change_index_by_polar_splits(self, phi_split_number):
        split_degree = 360 / phi_split_number
        color_val_label = "phi"
        self.type_of_change_label_list.append("phi")
        gcode_df_color = self.gcode_df.copy()
        color_change_val = gcode_df_color["phi"]
        gcode_df_color["delta"] = color_change_val % split_degree
        gcode_df_color["delta_val"] = color_change_val // split_degree

        self.calc_color_changes(gcode_df_color, color_val_label)
  
    def add_color_change_commands_to_gcode(self, g_lines_raw):

        color_index = 0
        

        for color_change in reversed(self.color_changes):

            color = self.color_change_order[color_index % len(self.color_change_order)]

            layer_nr = int(color_change.gcode_line_from_df.layer)

            color_command = self.get_color_change_commands(color)

            if self.reduce_pressure_filament_distance is not None:
                self.adjust_pressure_around_color_change(color_change, g_lines_raw)
            else:
                # retraction only if no pressure control
                color_command = self.add_retraction_commands(color_command, layer_nr)

            # add color commands and
            g_lines_raw[color_change.index:color_change.index] = color_command

            #print(color_change.index, color_command)

            color_index += 1

        self.gcode_result = g_lines_raw

    def write_results_to_file(self):
        all_lines = ["{}\n".format(line.gcode_str) for line in self.gcode_result]

        date_str = datetime.now().strftime("%m%d-%H:%M")

        new_file_name = "{}_colored{}_{}_{}".format(date_str, len(self.color_change_order), "-".join(self.type_of_change_label_list) , file_name)

        out = open(new_file_name, "w")
        out.writelines(all_lines)
        out.close()

        print(new_file_name)

In [72]:
class GcodeContainer:

    def __init__(self, gcode_raw, printer_setting_line_height, start_layer_height = 0.0):
        self.gcode_raw = gcode_raw
        self.lines_raw = GcodeParser(gcode, include_comments=True).lines
        self.start_layer_height = start_layer_height

        self.lines_start = None
        self.lines_end = None
        self.lines = None
        self.gcode_df = None

        self.model_center_point_xy = None
        self.model_height_mm = None
        self.model_layer_count = None

        self.printer_setting_line_height = printer_setting_line_height

        self.isCura = ';Generated with Cura_SteamEngine' in self.gcode_raw

        self.add_parameters_to_lines(self.lines_raw)

    def add_parameters_to_lines(self, lines_raw):
        self.lines = copy.deepcopy(lines_raw)
        
        section_type = None
        layer_nr = 0

        # add line number to params
        for ind, line in enumerate(self.lines):
            if "TYPE:" in line.gcode_str and self.isCura:
                section_type = line.gcode_str.split("TYPE:")[1]
            if "LAYER:" in line.gcode_str and self.isCura:
                layer_nr = line.gcode_str.split("LAYER:")[1]
            line.params["line_nr"] = ind
            line.params["layer_nr"] = int(layer_nr)
            line.params["section_type"] = section_type
            

    
    def separate_start_end(self, remove_first_line_count, limit_lines=None):
        
        gcode_lines = self.lines
        
        start_g_code = GcodeLine(command=(';', None), params={}, comment='END OF THE START GCODE')
        end_g_code = GcodeLine(command=(';', None), params={}, comment='START OF THE END GCODE')
        
        start_index = self.lines_raw.index(start_g_code) + remove_first_line_count
        end_index = self.lines_raw.index(end_g_code)

        # exclude bottom layer by adjusting the start_index
        start_index = self.exclude_bottom_layer(gcode_lines, start_index)
        
        self.lines_start = gcode_lines[:start_index]
        self.lines_end = gcode_lines[end_index:]
        self.lines = gcode_lines[start_index + 1: end_index]

        if limit_lines is not None:
            self.lines = self.lines[: limit_lines]
            self.lines_raw = self.lines_start + self.lines + self.lines_end
            for line in self.lines_raw:
                if 'line_nr' in line.params: del line.params['line_nr']


        for i, line in enumerate(self.lines_start[:20]):
            print(i, line.gcode_str)

        print("-------------")

        for i, line in enumerate(self.lines[:20]):
            print(i, line.gcode_str)

    def exclude_bottom_layer(self, gcode_lines, start_index):
        if self.start_layer_height > 0:
            # scan for layer
            start_layer_index = None
            for ind, line in enumerate(gcode_lines):
                if line.command == ("G", 1) and ind > start_index:
                    if "Z" in line.params:
                        if line.params["Z"] >= self.start_layer_height:
                            start_layer_index = ind
                            break
            if start_layer_index is not None:
                print("start found after bottom layers")
                print(line.gcode_str)
                print("------------------------")
                print([line.gcode_str for line in gcode_lines[start_layer_index -10: start_layer_index + 10]])
                start_index = start_layer_index
        return start_index

    def calc_g1_dataframe(self):
        # filter line extrusion command G1
        df_dict_list = [line.params for line in self.lines if line.command == ('G', 1)]
        gcode_df = pd.DataFrame(df_dict_list)

        # fill null with absolute (coordinates)
        gcode_df.E = gcode_df.E.fillna(0)
        gcode_df.fillna(method="ffill", inplace=True)
        gcode_df = gcode_df.fillna(method="bfill")

        #calculate
        
        # line_number
        if not self.isCura:
            gcode_df["layer_nr"] = (gcode_df["Z"] // line_height)

        # distance
        gcode_df["distance_xy"] = np.sqrt((gcode_df.X.shift(1) - gcode_df.X) ** 2 + (gcode_df.Y.shift(1) - gcode_df.Y) ** 2)
        #gcode_df["e_ratio_distance"] = (gcode_df.E / gcode_df.distance_xy )
        gcode_df["distance_xy_cumsum"] = gcode_df["distance_xy"].cumsum()

        center_point = (gcode_df.X.mean(), gcode_df.Y.mean())
        self.model_center_point_xy = center_point
        self.model_height_mm = gcode_df.Z.max()
        self.model_layer_count = int(round(self.model_height_mm / self.printer_setting_line_height))
        print("CenterPointXY",center_point)

        # calculate polar coordinates
        gcode_df["rho"] = np.sqrt((gcode_df["X"]- center_point[0])**2 + (gcode_df["Y"]-center_point[1])**2)
        gcode_df["phi"] = np.degrees(np.arctan2(gcode_df["X"]- center_point[0], gcode_df["Y"]-center_point[1]))

        self.gcode_df = gcode_df

In [73]:

#file_name = 'ConeThicker3.gcode'
file_name = 'CubeSpiralWithoutButtom.gcode'
line_height = 0.3

remove_first_line_count = 0 #5

# load gcode file
with open(file_name) as f:
    gcode = f.read()

gcode = GcodeContainer(gcode,
                       start_layer_height=0,
                       printer_setting_line_height=line_height)

gcode.separate_start_end(remove_first_line_count, limit_lines=None)

gcode.calc_g1_dataframe()

gcode.gcode_df[["line_nr", "Z", "X", "Y", "E", "F","layer_nr", "section_type", "distance_xy_cumsum", "phi"]][:100]

0 ; FLAVOR:Marlin
1 ; TIME:183
2 ; Filament used: 0.329837m
3 ; Layer height: 0.3
4 ; MINX:95.71
5 ; MINY:115.697
6 ; MINZ:0.3
7 ; MAXX:124.298
8 ; MAXY:144.297
9 ; MAXZ:19.957
10 ; ZOFFSETPROCESSED
11 ; Generated with Cura_SteamEngine 99.9.1-Arachne_engine_beta
12 M140 S70 line_nr12 layer_nr0 section_typeNone
13 M105 line_nr13 layer_nr0 section_typeNone
14 M190 S70 line_nr14 layer_nr0 section_typeNone
15 M104 S240 line_nr15 layer_nr0 section_typeNone
16 M105 line_nr16 layer_nr0 section_typeNone
17 M109 S240 line_nr17 layer_nr0 section_typeNone
18 M82 line_nr18 layer_nr0 section_typeNone ; absolute extrusion mode
19 G28 line_nr19 layer_nr0 section_typeNone
-------------
0 M83 line_nr38 layer_nr0 section_typeNone ; relative extrusion mode
1 G1 F2100 E-3 line_nr39 layer_nr0 section_typeNone
2 ; LAYER_COUNT:66
3 ; LAYER:0
4 M107 line_nr42 layer_nr0 section_typeNone
5 G0 F9000 X100.775 Y119.082 Z0.25 line_nr43 layer_nr0 section_typeNone ; adjusted by z offset
6 G92 Z0.3 line_nr44 layer_nr0

,line_nr,Z,X,Y,E,F,layer_nr,section_type,distance_xy_cumsum,phi
0,39,0.303,101.323,119.310,-3.00000,2100.0,0,SKIRT,NaN,-140.912258
1,47,0.303,101.323,119.310,3.00000,2100.0,0,SKIRT,0.000000,-140.912258
2,48,0.303,101.323,119.310,0.02031,1500.0,0,SKIRT,0.000000,-140.912258
3,49,0.303,101.873,118.885,0.05462,1500.0,0,SKIRT,0.695072,-143.801044
4,50,0.303,102.106,118.718,0.02253,1500.0,0,SKIRT,0.981739,-144.992323
...,...,...,...,...,...,...,...,...,...,...
95,141,0.303,103.073,141.908,0.05183,1500.0,0,SKIRT,59.913830,-30.267794
96,142,0.303,102.874,141.786,0.01834,1500.0,0,SKIRT,60.147250,-31.238757
97,143,0.303,102.167,141.324,0.06636,1500.0,0,SKIRT,60.991817,-34.753429
98,144,0.303,101.937,141.161,0.02215,1500.0,0,SKIRT,61.273719,-35.926511


In [79]:
class WobbleGenerator:

    def __init__(self, gcode_container):

        self.gcode_container = gcode_container
        self.gcode_df = gcode_container.gcode_df

        self.layer_parameter = []

        self.analyze_layer()

    def analyze_layer(self):
        
        for layer_nr in range(self.gcode_container.model_layer_count):
            
            lines = self.gcode_container.gcode_df[self.gcode_df["layer_nr"] == 0]
            layer_length = lines["distance_xy"].sum()
            self.layer_parameter.append({"length": layer_length})

    def generate_wooble(self, wobble_count, wobble_amplitude, wobble_segments, toggle_multiplier=0):

        
        for layer_nr in range(self.gcode_container.model_layer_count):
            
            wobble_length = self.layer_parameter[layer_nr]["length"] / wobble_count
            wobble_segment_length = wobble_length/wobble_segments
            
            lines = self.gcode_container.gcode_df[self.gcode_df["layer_nr"] == 0]

            # layer lines aufteilen in wobble segmente

            # sinus signal addieren

            for line in lines:
                line[""]
            
            break



gen = WobbleGenerator(gcode)

gen.generate_wooble(wobble_count=50, wobble_amplitude=2.0, wobble_segments=16)

gcode.gcode_df[:100]

,F,E,line_nr,layer_nr,section_type,X,Y,Z,distance_xy,distance_xy_cumsum,rho,phi
0,2100.0,-3.00000,39,0,SKIRT,101.323,119.310,0.303,NaN,NaN,13.787107,-140.912258
1,2100.0,3.00000,47,0,SKIRT,101.323,119.310,0.303,0.000000,0.000000,13.787107,-140.912258
2,1500.0,0.02031,48,0,SKIRT,101.323,119.310,0.303,0.000000,0.000000,13.787107,-140.912258
3,1500.0,0.05462,49,0,SKIRT,101.873,118.885,0.303,0.695072,0.695072,13.787725,-143.801044
4,1500.0,0.02253,50,0,SKIRT,102.106,118.718,0.303,0.286667,0.981739,13.787861,-144.992323
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1500.0,0.05183,141,0,SKIRT,103.073,141.908,0.303,0.659619,59.913830,13.774452,-30.267794
96,1500.0,0.01834,142,0,SKIRT,102.874,141.786,0.303,0.233420,60.147250,13.771365,-31.238757
97,1500.0,0.06636,143,0,SKIRT,102.167,141.324,0.303,0.844567,60.991817,13.768900,-34.753429
98,1500.0,0.02215,144,0,SKIRT,101.937,141.161,0.303,0.281902,61.273719,13.768974,-35.926511


In [10]:
color_labels = ["blue", "white", "red"]

color_handler = ColorChangeHandler(gcode,
                                   color_change_order=[(1,0,0), (0,0,1)],
                                   reduce_pressure_filament_distance=1,
                                   negative_pressure_reduction_factor=1)
                                   #rectraction_length=[4, 3, 2, 1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
                                   #refill_retraction_length=[4, 3, 2, 1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
                                   #retraction_speed=1800)
#color_handler.calc_color_change_index_by_polar_splits(2)
color_handler.calc_color_change_index_by_layer_count(1)
color_handler.add_color_change_commands_to_gcode(gcode.lines_raw)

color_handler.write_results_to_file()

color_handler.gcode_result[:1000]

#color_handler.gcode_df_color[["phi", "delta_val", "delta", "color_change_index"]]

# prime instead of refill

layer
0717-19:11_colored2_layer_ConeThicker3.gcode


[GcodeLine(command=(';', None), params={}, comment='FLAVOR:Marlin'),
 GcodeLine(command=(';', None), params={}, comment='TIME:1478'),
 GcodeLine(command=(';', None), params={}, comment='Filament used: 3.8675m'),
 GcodeLine(command=(';', None), params={}, comment='Layer height: 0.4'),
 GcodeLine(command=(';', None), params={}, comment='MINX:76.12'),
 GcodeLine(command=(';', None), params={}, comment='MINY:96.116'),
 GcodeLine(command=(';', None), params={}, comment='MINZ:0.3'),
 GcodeLine(command=(';', None), params={}, comment='MAXX:143.896'),
 GcodeLine(command=(';', None), params={}, comment='MAXY:163.889'),
 GcodeLine(command=(';', None), params={}, comment='MAXZ:70.112'),
 GcodeLine(command=(';', None), params={}, comment='Generated with Cura_SteamEngine 99.9.1-Arachne_engine_beta'),
 GcodeLine(command=('M', 140), params={'S': 70}, comment=''),
 GcodeLine(command=('M', 105), params={}, comment=''),
 GcodeLine(command=('M', 190), params={'S': 70}, comment=''),
 GcodeLine(command=('M